In [1]:
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
df = pd.read_csv('data/maindata.csv', dtype={'price_string_unf': 'object'})

# Dask is faster for larger datasets but it doesnt show inline output, so i've not used it here
daskdf = dd.read_csv('data/maindata.csv', dtype={'price_string_unf': 'object'})

In [3]:
df.shape

(13730000, 6)

In [4]:
df.describe()

,uuid,price_string,price_string_unf,product_type,level_1,category
count,13730000,5510000,310000,13720000,13680000,13720000
unique,1373,248,26,321,595,13
top,638744a4-b0ae-4166-8cb6-5c063c862036,$0.00,$6.75,U2NydWJzIGFuZCBjbGVhbmluZyBjbG90aHM,U3dlYXRzaGlydA,Q2xvdGhpbmcgYW5kIEFjY2Vzc29yaWVz
freq,10000,240000,30000,200000,90000,3910000


## *Question 1*
Products without prices

In [5]:
df.head()

,uuid,price_string,price_string_unf,product_type,level_1,category
0,638744a4-b0ae-4166-8cb6-5c063c862036,NaN,NaN,TGFwdG9wIENvdmVycy9CYWdz,TGFwdG9wIENhc2U,RWxlY3Ryb25pY3M
1,ab313969-02cc-48b2-9daf-0054efb70b92,NaN,NaN,QmFraW5nIEN1cHMgYW5kIE1hdHM,QmFraW5nIE1hdHMgLyBCYWtpbmcgZGlzaA,a2l0Y2hpbmcgYW5kIGRpbmluZw
2,acbd66ff-79f8-467a-91f9-108a45af5626,$19.95,NaN,R3VtbWllcyB2aXRhbWlucyBhbmQgbWluZXJhbHMgZm9yIG...,SW1tdW5pdHkgZ3VtbWllcw,SGVhbHRo
3,963915d6-b2e3-4098-b242-9410593cf205,$92.00,NaN,U2VydW1z,RmFjZSBTZXJ1bQ,YmVhdXR5IGFuZCBwZXJzb25hbCBjYXJl
4,b5b68f3c-b1e0-40e5-8ee5-e2f7236c1daf,11.50,NaN,RWF0aW5nIFV0ZW5zaWxzL0N1dGxlcnk,Q2hvcHN0aWNrcw,a2l0Y2hpbmcgYW5kIGRpbmluZw


In [6]:
# Faster, just count
df['price_string'].isna().value_counts()

True     8220000
False    5510000
Name: price_string, dtype: int64

In [7]:
# Slower, returns data. This df has all products without price_string
noprices = df[df['price_string'].isna()]
noprices.shape

(8220000, 6)

### There are 8,220,000 products without prices
All products without price are stored in noprices.

## *Question 2*
Count of products without prices and with prices in each Product Type, Category,
Level 1

### Product Type

In [8]:
# Number of products which have a price, grouped by product type
grpby_prodtype = df.groupby('product_type')

In [9]:
hasprice_prodtype = grpby_prodtype.count()
hasprice_prodtype.rename(columns={"price_string": "hasprice"}, inplace=True)
hasprice_prodtype.hasprice

product_type
Q0FUIExJVFRFUg                                  0
Q29mZmVlIEJlYW5z                            30000
Q29mZmVlIEZpbHRlcnM                             0
Q29uZGl0aW9uZXIgSGFpcg                      10000
Q29va2llcw                                      0
                                            ...  
cGV0IEJsYW5rZXQ                             10000
cGV0IFN0YWluIGFuZCBvZG9yIGVsaW1pbmF0b3I         0
d29tZW5zIE91dGVyd2Vhcg                     130000
d29tZW5zIFdhbGxldA                          20000
d29tZW5zIHN3ZWF0ZXJz                        30000
Name: hasprice, Length: 321, dtype: int64

In [10]:
totalsize_prodtype = grpby_prodtype.size()
noprice_prodtype = totalsize_prodtype - hasprice_prodtype.hasprice
noprice_prodtype

product_type
Q0FUIExJVFRFUg                             30000
Q29mZmVlIEJlYW5z                           50000
Q29mZmVlIEZpbHRlcnM                        10000
Q29uZGl0aW9uZXIgSGFpcg                     20000
Q29va2llcw                                 30000
                                           ...  
cGV0IEJsYW5rZXQ                            10000
cGV0IFN0YWluIGFuZCBvZG9yIGVsaW1pbmF0b3I    10000
d29tZW5zIE91dGVyd2Vhcg                     20000
d29tZW5zIFdhbGxldA                         10000
d29tZW5zIHN3ZWF0ZXJz                       90000
Length: 321, dtype: int64

### Category

In [11]:
grpby_category = df.groupby('category')

In [12]:
hasprice_category = df.groupby('category').count()
hasprice_category.rename(columns={"price_string": "hasprice"}, inplace=True)
hasprice_category.hasprice

category
Q2xvdGhpbmcgYW5kIEFjY2Vzc29yaWVz       1860000
QXJ0IHN1cHBsaWVz                        190000
QmFieWNhcmU                              90000
RWxlY3Ryb25pY3M                         110000
SG91c2Vob2xkIGFuZCBDbGVhbmluZw          700000
SGVhbHRo                                290000
VG95cyBhbmQgR2FtZXM                     190000
VG9vbHMgYW5kIGhvbWUgaW1wcm92ZW1lbnQ     200000
YmVhdXR5IGFuZCBwZXJzb25hbCBjYXJl        750000
Z3JvY2VyaWVz                            480000
a2l0Y2hpbmcgYW5kIGRpbmluZw              340000
b2ZmaWNlIHByb2R1Y3Rz                    150000
cGV0IHN1cHBsaWVz                        160000
Name: hasprice, dtype: int64

In [13]:
totalsize_category = grpby_category.size()
noprice_category =  totalsize_category- hasprice_category.hasprice
noprice_category

category
Q2xvdGhpbmcgYW5kIEFjY2Vzc29yaWVz       2050000
QXJ0IHN1cHBsaWVz                        130000
QmFieWNhcmU                             570000
RWxlY3Ryb25pY3M                         210000
SG91c2Vob2xkIGFuZCBDbGVhbmluZw          870000
SGVhbHRo                                700000
VG95cyBhbmQgR2FtZXM                     600000
VG9vbHMgYW5kIGhvbWUgaW1wcm92ZW1lbnQ     320000
YmVhdXR5IGFuZCBwZXJzb25hbCBjYXJl        720000
Z3JvY2VyaWVz                           1130000
a2l0Y2hpbmcgYW5kIGRpbmluZw              250000
b2ZmaWNlIHByb2R1Y3Rz                     80000
cGV0IHN1cHBsaWVz                        580000
dtype: int64

### Level 1


In [14]:
grpby_level1 = df.groupby('level_1')
hasprice_level1 = df.groupby('level_1').count()


In [15]:
hasprice_level1 = df.groupby('level_1').count()
hasprice_level1.rename(columns={"price_string": "hasprice"}, inplace=True)
hasprice_level1.hasprice

level_1
MTAgQ3Vw              10000
MjAgQ3Vw              10000
MjAgUGludHM           10000
MzAgQ3Vw              10000
MzAtIDM1IFBpbnRz      10000
                      ...  
d29tZW5zIGJvb3Rz      20000
d29tZW5zIGhlZWxz      10000
eW9nYSBCbG9ja3M       20000
eW9nYSBib2xzdGVycw    10000
eW9nYSBzdHJhcHM       30000
Name: hasprice, Length: 595, dtype: int64

In [16]:
totalsize_level1 = grpby_level1.size()
noprice_level1 = totalsize_level1 - hasprice_level1.hasprice
noprice_level1

level_1
MTAgQ3Vw                  0
MjAgQ3Vw                  0
MjAgUGludHM           20000
MzAgQ3Vw                  0
MzAtIDM1IFBpbnRz      10000
                      ...  
d29tZW5zIGJvb3Rz      20000
d29tZW5zIGhlZWxz          0
eW9nYSBCbG9ja3M       10000
eW9nYSBib2xzdGVycw    10000
eW9nYSBzdHJhcHM           0
Length: 595, dtype: int64

## *Question 3*
Correct Product Prices in the correct format (eg: $56) wherever possible and
separate them into currency and value columns

In [17]:
df_with_prices = pd.DataFrame(df['price_string'].dropna())

In [18]:
def add_doll(price):
    if price[0]!='$':
        return '$'+price
    return price

In [19]:
df_with_prices['price'] = df_with_prices['price_string'].apply(add_doll)

In [20]:
regex_flag = df['price_string'].str.contains('\d+\$')

In [21]:
tempe = regex_flag.fillna(False)

In [22]:
df[tempe]

,uuid,price_string,price_string_unf,product_type,level_1,category


In [23]:
df_with_prices['value'] = df_with_prices['price'].apply(lambda x: x.replace('$', ''))

In [24]:
df_with_prices

#TODO: Do this with main dataset

,price_string,price,value
2,$19.95,$19.95,19.95
3,$92.00,$92.00,92.00
4,11.50,$11.50,11.50
6,$24.99,$24.99,24.99
7,$148.00,$148.00,148.00
...,...,...,...
13729987,$32.99,$32.99,32.99
13729989,$34.00,$34.00,34.00
13729994,$86.00,$86.00,86.00
13729996,$69.99,$69.99,69.99


## Question 4
List out the categories with average price of product.

,uuid,price_string,price_string_unf,product_type,level_1
category,,,,,
Q2xvdGhpbmcgYW5kIEFjY2Vzc29yaWVz,3910000,1860000,120000,3910000,3910000
QXJ0IHN1cHBsaWVz,320000,190000,0,320000,310000
QmFieWNhcmU,660000,90000,80000,660000,660000
RWxlY3Ryb25pY3M,320000,110000,0,320000,320000
SG91c2Vob2xkIGFuZCBDbGVhbmluZw,1570000,700000,0,1570000,1570000
SGVhbHRo,990000,290000,30000,990000,990000
VG95cyBhbmQgR2FtZXM,790000,190000,20000,790000,790000
VG9vbHMgYW5kIGhvbWUgaW1wcm92ZW1lbnQ,520000,200000,0,520000,500000
YmVhdXR5IGFuZCBwZXJzb25hbCBjYXJl,1470000,750000,0,1470000,1470000
